# Own parameterised AI - run experiments¶

Runs a series of experiments of OpenTTDLab, running https://github.com/michalc/ParameterisedAI for a varying number of buses.

Results are saved to 02_own_parameterised_ai_01_raw.csv.

In [ ]:
!python -m pip install OpenTTDLab==0.0.72 pandas==2.2.0

In [2]:
from openttdlab import run_experiments, remote_file, bananas_ai_library

def process_result(result):
    return [{
        # It's slightly awkward right now to get at the original AI params
        'max_buses': result['experiment']['ais'][0][1][0][1],
        'seed': result['experiment']['seed'],
        'date': result['date'],
        'money': result['chunks']['PLYR']['0']['money'],
    }]

results = run_experiments(
    openttd_version='13.4',
    opengfx_version='7.1',
    experiments=(
        {
             'seed': seed,
             'ais': (
                remote_file(
                    'https://github.com/michalc/ParameterisedAI/archive/d3ac662b47267ed4fa84a5b3997c020ef140f1e2.tar.gz',
                    ai_name='ParameterisedAI',
                    ai_params=(
                        ('maximum_buses', maximum_buses),
                    ),
                ),
             ),
            'days': 366 * 50 + 1,
        }
        for maximum_buses in [1, 2, 4, 8, 16]
        for seed in range(1, 51)
    ),
    ai_libraries=(
        bananas_ai_library('5046524f', 'Pathfinder.Road', md5='999de61cd3a10680b4ff91547299dc53'),
        bananas_ai_library('4752412a', 'Graph.AyStar', md5='f385497c3c922bfd9f61e1bc33b3a4dc'),
        bananas_ai_library('51554248', 'Queue.BinaryHeap', md5='8ce55e1397e9c51f8032c3a8a29e9cf5'),
    ),
    result_processor=process_result,
)

Output()

In [3]:
import pandas as pd

df = pd.DataFrame(results)
df.to_csv('02_own_parameterised_ai_results_01_raw.csv', index=False)